In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset APAC
df = pd.read_excel(r"C:\Users\sabri\APACTeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,win,GRACESBLAZE,SAGETTA,657,1,0,1,4,1,25.000000,5,5,1.000000
1,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,loss,SAGETTA,GRACESBLAZE,426,0,0,1,1,4,0.000000,5,5,0.250000
2,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,loss,BROKOLI,GLADIATORS,644,1,1,3,6,8,16.666667,0,8,0.375000
3,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,win,GLADIATORS,BROKOLI,1438,4,4,4,8,6,50.000000,8,0,0.666667
4,013f1db9-799d-40d1-80f7-92405243561a,APAC P-H N55 vs GG G2 2023-02-12.18.22,Forbidden Temple,2023-02-12,win,GLADIATORS,NIMMT55,1484,5,5,3,9,8,55.555556,4,6,0.375000


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,win,GRACESBLAZE,SAGETTA,657,1,0,1,4,1,25.000000,5,5,1.000000
1,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,loss,SAGETTA,GRACESBLAZE,426,0,0,1,1,4,0.000000,5,5,0.250000
2,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,loss,BROKOLI,GLADIATORS,644,1,1,3,6,8,16.666667,0,8,0.375000
3,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,win,GLADIATORS,BROKOLI,1438,4,4,4,8,6,50.000000,8,0,0.666667
4,013f1db9-799d-40d1-80f7-92405243561a,APAC P-H N55 vs GG G2 2023-02-12.18.22,Forbidden Temple,2023-02-12,win,GLADIATORS,NIMMT55,1484,5,5,3,9,8,55.555556,4,6,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,ff6f8b77-0fb5-48b2-8413-544cade301cd,APAC G LE vs ELV G4 2023-02-25.16.01,Utopia Coliseum (Dusk),2023-02-25,loss,ELUVA,LYCUS EMPIRE,721,0,0,7,1,14,0.000000,4,4,0.500000
2492,ff8fd82c-2fc5-41cf-b1f6-4443c6d72020,APAC Z N55 vs GS G1 2022-11-05.18.39,Mannfield (Night),2022-11-05,loss,GOD SPEED,NIMMT55,777,1,0,2,4,6,25.000000,3,1,0.333333
2493,ff8fd82c-2fc5-41cf-b1f6-4443c6d72020,APAC Z N55 vs GS G1 2022-11-05.18.39,Mannfield (Night),2022-11-05,win,NIMMT55,GOD SPEED,1028,2,2,3,6,4,33.333333,1,3,0.750000
2494,ff9883a7-7375-46e2-a3fe-77dc1b839291,APAC Z N55 vs DTN G2 2022-10-22.22.27,Aquadome,2022-10-22,win,NIMMT55,DETONATOR,1077,3,1,3,8,4,37.500000,2,5,0.750000


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,win,GRACESBLAZE,SAGETTA,657,1,0,1,4,1,25.000000,5,5,1.000000
1,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,loss,SAGETTA,GRACESBLAZE,426,0,0,1,1,4,0.000000,5,5,0.250000
2,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,loss,BROKOLI,GLADIATORS,644,0,1,3,6,8,16.666667,0,8,0.375000
3,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,win,GLADIATORS,BROKOLI,1438,0,4,4,8,6,50.000000,8,0,0.666667
4,013f1db9-799d-40d1-80f7-92405243561a,APAC P-H N55 vs GG G2 2023-02-12.18.22,Forbidden Temple,2023-02-12,win,GLADIATORS,NIMMT55,1484,0,5,3,9,8,55.555556,4,6,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,ff6f8b77-0fb5-48b2-8413-544cade301cd,APAC G LE vs ELV G4 2023-02-25.16.01,Utopia Coliseum (Dusk),2023-02-25,loss,ELUVA,LYCUS EMPIRE,721,0,0,7,1,14,0.000000,4,4,0.500000
2492,ff8fd82c-2fc5-41cf-b1f6-4443c6d72020,APAC Z N55 vs GS G1 2022-11-05.18.39,Mannfield (Night),2022-11-05,loss,GOD SPEED,NIMMT55,777,1,0,2,4,6,25.000000,3,1,0.333333
2493,ff8fd82c-2fc5-41cf-b1f6-4443c6d72020,APAC Z N55 vs GS G1 2022-11-05.18.39,Mannfield (Night),2022-11-05,win,NIMMT55,GOD SPEED,1028,0,2,3,6,4,33.333333,1,3,0.750000
2494,ff9883a7-7375-46e2-a3fe-77dc1b839291,APAC Z N55 vs DTN G2 2022-10-22.22.27,Aquadome,2022-10-22,win,NIMMT55,DETONATOR,1077,2,1,3,8,4,37.500000,2,5,0.750000


In [5]:
#  Checando se há empates nos resultados e removendo-os
df_draw = df[df['result']=='draw']
df = df.drop(df_draw.index)

In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,GRACESBLAZE,SAGETTA,657,1,0,1,4,1,25.000000,5,5,1.000000,0
1,00ac273c-7275-41ae-b80c-7aabfeacb1a6,APAC S GB vs SGT G1 2023-05-13.19.17,Mannfield (Night),2023-05-13,SAGETTA,GRACESBLAZE,426,0,0,1,1,4,0.000000,5,5,0.250000,1
2,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,BROKOLI,GLADIATORS,644,0,1,3,6,8,16.666667,0,8,0.375000,1
3,012b84da-3406-40dc-bf35-3f8d784c9799,APAC S GG vs BRO G4 2023-02-25.18.01,Utopia Coliseum (Dusk),2023-02-25,GLADIATORS,BROKOLI,1438,0,4,4,8,6,50.000000,8,0,0.666667,0
4,013f1db9-799d-40d1-80f7-92405243561a,APAC P-H N55 vs GG G2 2023-02-12.18.22,Forbidden Temple,2023-02-12,GLADIATORS,NIMMT55,1484,0,5,3,9,8,55.555556,4,6,0.375000,0


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2244, 12) (248, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(248, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1683, 9), (561, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89       271
           1       0.89      0.91      0.90       290

    accuracy                           0.90       561
   macro avg       0.90      0.90      0.90       561
weighted avg       0.90      0.90      0.90       561



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

cv = KFold(n_splits=10, shuffle=True, random_state=10)

In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=20,
                           n_estimators=300)


In [19]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 20, learning_rate = 0.05, n_estimators = 300)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       267
           1       0.91      0.89      0.90       294

    accuracy                           0.89       561
   macro avg       0.89      0.90      0.89       561
weighted avg       0.90      0.89      0.89       561
               precision    recall  f1-score   support

           0       0.85      0.90      0.88       124
           1       0.90      0.85      0.87       124

    accuracy                           0.88       248
   macro avg       0.88      0.88      0.87       248
weighted avg       0.88      0.88      0.87       248



In [20]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       855
           1       0.94      0.95      0.95       828

    accuracy                           0.95      1683
   macro avg       0.95      0.95      0.95      1683
weighted avg       0.95      0.95      0.95      1683



In [21]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.02486363151170133, 0.052569070797281785, 0.3634362751179402, 0.08465313282624344, 0.1303847811667098, 0.10128527071443852, 0.002910997971635384, 0.0034940555948270737, 0.2364027842992224]
